In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cap = cv2.VideoCapture(0)

cap.set(3, 640)
cap.set(3, 480)

True

In [3]:
def getContours(img, original_img):
    contours, hierachy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        if area > 100000:
            # print(area) # to find right area
            cv2.drawContours(original_img, cnt, -1, (0, 255, 0), 2)
            
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            ax = approx.item(0)
            ay = approx.item(1)
            bx = approx.item(2)
            by = approx.item(3)
            cx = approx.item(4)
            cy = approx.item(5)
            dx = approx.item(6)
            dy = approx.item(7)
            
            width, height= 702, 702 # divisible by 9 to capture each cell

            pts1 = np.float32([[bx, by], [ax, ay], [cx, cy], [dx, dy]])
            pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            img_perspective = cv2.warpPerspective(original_img, matrix, (width, height))
            img_corners = cv2.cvtColor(img_perspective, cv2.COLOR_BGR2GRAY)
            
            # turn into a binary image
            for x in range(0, 702):
                for y in range(0, 702):
                    if img_corners[x][y]<180:
                        img_corners[x][y]=0
                    else:
                        img_corners[x][y]=255
            
            cv2.imshow('Corners', img_corners)
            
            pickImages(grid266, img_corners)
            
#             crop_value = 10
        
#             for y in range(1, 9):
#                 for x in range(1, 9):
#                     plt.imshow(img_corners[y*78-78+crop_value:y*78-crop_value, x*78-78+crop_value:x*78-crop_value], cmap='gray')
#                     plt.show()
                        

In [4]:
def pickImages(grid,Img):
     
    DATADROP = "IMG//"
    crop_val = 10
    
    
    listNum = []
    for i in range(0, 9):
        for j in range(0, 9):
            if grid[i][j] not in listNum: 
                print(grid[i][j])
                listNum.append(grid[i][j])
                J = j+1
                I = i+1
                cell = Img[I*78 - 78 + crop_val : I*78 - crop_val , J*78 - 78 + crop_val : J*78 - crop_val]
                
                contours,hierachy = cv2.findContours(cell,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
                print(len(contours))
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if area > 600:
                        #cv2.drawContours(original_img,cnt,-1,(0,255,0),2)
                        #print(area)
                        peri = cv2.arcLength(cnt,True)
                        approx = cv2.approxPolyDP(cnt,0.02*peri,True)

                cv2.imwrite(DATADROP + str(grid[i][j])+ "//IMG_{}.png".format((i+1)*(j+1)),cell)
                print(DATADROP + str(grid[i][j])+ "//IMG_{}.png".format((i+1)*(j+1)))

In [5]:
while True:
    success, img = cap.read()
    
    # turning the original image to canny.
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),3)
    imgCanny = cv2.Canny(imgBlur,50,50)
    img_copy = img.copy()
    
    
    grid266   = [[3,2,0,0,1,0,7,8,0], 
                 [0,0,6,4,0,7,0,1,0],
                 [0,0,9,0,0,3,0,5,2],
                 [0,0,0,0,0,0,9,0,4],
                 [0,9,0,7,6,4,8,3,0],
                 [7,0,0,5,0,8,0,6,1],
                 [9,6,2,0,7,0,5,4,0],
                 [8,0,0,3,4,0,0,0,0],
                 [0,0,0,2,0,6,0,0,8]]
    
    img_contours = img.copy()
    
    # getting the countours
    getContours(imgCanny, img_copy)
    
    cv2.imshow('Webcam',img_copy)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        cap.release() #turn off cam
        cv2.destroyAllWindows()
        break
    

3
1
IMG//3//IMG_1.png
2
1
IMG//2//IMG_2.png
0
1
IMG//0//IMG_3.png
1
1
IMG//1//IMG_5.png
7
1
IMG//7//IMG_7.png
8
1
IMG//8//IMG_8.png
6
1
IMG//6//IMG_6.png
4
1
IMG//4//IMG_8.png
9
1
IMG//9//IMG_9.png
5
1
IMG//5//IMG_24.png


True

In [ ]:
CATEGORIES = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"] 
DATADIR = "IMG//"  
SAVE_DIR = "dataset//"  

os.makedirs(SAVE_DIR, exist_ok=True)

def data_augmentation():
    for category in CATEGORIES:  
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category) 

        save_path = os.path.join(SAVE_DIR, category)
        os.makedirs(save_path, exist_ok=True) 

        for img_filename in os.listdir(path): 
            if '.png' not in img_filename:
                continue
            try:
                img_array = cv2.imread(os.path.join(path, img_filename), cv2.IMREAD_GRAYSCALE)
                img_canny = cv2.Canny(img_array, 50, 50)
                contours, hierachy = cv2.findContours(img_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    
                    if area > 5:
                        
                        kernel = np.ones((5, 5), np.uint8)
                        
                        for blur_value in range(-30, 30):
                            
                            img = cv2.GaussianBlur(img_array, (7, 7), blur_value)

                            cv2.imwrite(os.path.join(save_path, f"blur_{blur_value}_{img_filename}"), img)

                            img_erosion = cv2.erode(img, kernel, iterations=1)
                            cv2.imwrite(os.path.join(save_path, f"erosion1_{blur_value}_{img_filename}"), img_erosion)

                            img_dilation = cv2.dilate(img, kernel, iterations=1)
                            cv2.imwrite(os.path.join(save_path, f"dilation1_{blur_value}_{img_filename}"), img_dilation)
                            

            except Exception as e:
                print(f"Error processing {img_filename}: {e}")

data_augmentation()
